In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
pd.set_option('display.float_format', '{:.6f}'.format)

# Import Model

In [ ]:
jupyter_dir = os.path.abspath(str(Path()))
jupyter_dir

In [ ]:
models_path = os.path.join(jupyter_dir,'trained_models','trained_models')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

In [ ]:
model = models[1]
model

In [ ]:
model_path = os.path.join(models_path,model)

In [ ]:
model_files = os.listdir(model_path)
model_files

In [ ]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [ ]:
import joblib

In [ ]:
model_fit = joblib.load(os.path.join(model_path,[file for file in model_files if file.endswith('.pkl')][0]))

In [ ]:
with open(spec_path, "r") as specs:
    for line in specs:
        print(line.strip())

In [ ]:
model_files

In [ ]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")

outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outsample') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]

insample_path = os.path.join(model_path,[file for file in model_files if file.find('insample') != -1][0])
insample = pd.read_csv(insample_path).iloc[:,1:]

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    print(
        f"\n         {'#'*(len(w)+5)}"
        f"\n         # {w}s #"
        f"\n         {'#'*(len(w)+5)}"
    )
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    print(f"RMSE:\n{RMSE}")
    print()
    errors['RMSE'] = RMSE
    


    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    print(f"MAE:\n{MAE}")
    print()
    errors['MAE'] = MAE
    errors_bypayoff[w] = errors

In [ ]:
max_MAE = max(errors['MAE'].values.flatten())
max_RMSE = max(errors['RMSE'].values.flatten())
plot_height = max(max_RMSE,max_MAE)*1.05
maturities = errors['MAE'].index.astype(str).tolist()

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15,10))
bar_width = 0.4
i = 0
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names:

        MAE = errors['MAE'][b]
        RMSE = errors['RMSE'][b]
        x_positions = range(len(maturities))
        axs[i // 4, i % 4].grid(True,which='both', linestyle='--', color='gray')
        axs[i // 4, i % 4].bar(
            [x - 0.5 * bar_width for x in x_positions], MAE, width=bar_width, color='green',label="MAE"
        )
        
        axs[i // 4, i % 4].bar(
            [x + 0.5 * bar_width for x in x_positions], RMSE, width=bar_width, color='purple', label="RMSE"
        )
        axs[i // 4, i % 4].set_title(f'{b} {w}s')
        axs[i // 4, i % 4].set_xticks(x_positions)
        axs[i // 4, i % 4].set_xticklabels(maturities)
        axs[i // 4, i % 4].set_ylim(0, plot_height)
        axs[i // 4, i % 4].legend()
        i += 1

fig.supxlabel('maturities')

plt.tight_layout()
plt.show()

In [ ]:
types  = ['DownOut', 'DownIn', 'UpOut', 'UpIn']

In [ ]:
for b in types:
    print(f"\n{b}")
    print(by_wbt.loc[('call',b,slice(None)),:].shape[0])

In [ ]:
types  = ['DownOut', 'DownIn', 'UpOut', 'UpIn']
for b in types:
    print(f"\n{b}")
    print(by_wbt.loc[('put',b,slice(None)),:].shape[0])

In [ ]:
outsample.loc[:,'security_type'] = outsample['w'] + outsample['barrier_type_name']

In [ ]:
down_calls = outsample.copy()[
    (
        (outsample['w'] == 'call')
        &
        (outsample['updown'] =='Down')
    )
]
observed_price = np.sum(down_calls['barrier_price'])
outofsample_prediction = np.sum(down_calls['outofsample_prediction'])
print(f"\n{observed_price}\n{outofsample_prediction}\n")
RMSE = float(np.sqrt(np.mean(np.array(down_calls['outofsample_prediction']-down_calls['barrier_price'])**2)))
print(RMSE)
MAE = float(np.average(np.abs(np.array(down_calls['outofsample_prediction']-down_calls['barrier_price']))))
MAE

In [ ]:
down_puts = outsample.copy()[
    (
        (outsample['w'] == 'put')
        &
        (outsample['updown'] =='Down')
    )
]
observed_price = np.sum(down_puts['barrier_price'])
outofsample_prediction = np.sum(down_puts['outofsample_prediction'])
print(f"\n{observed_price}\n{outofsample_prediction}\n")
RMSE = float(np.sqrt(np.mean(np.array(down_puts['outofsample_prediction']-down_puts['barrier_price'])**2)))
print(RMSE)
MAE = float(np.average(np.abs(np.array(down_puts['outofsample_prediction']-down_puts['barrier_price']))))
MAE

In [ ]:
up_calls = outsample.copy()[
    (
        (outsample['w'] == 'call')
        &
        (outsample['updown'] =='Up')
    )
]
observed_price = np.sum(up_calls['barrier_price'])
outofsample_prediction = np.sum(up_calls['outofsample_prediction'])
print(f"\n{observed_price}\n{outofsample_prediction}\n")
RMSE = float(np.sqrt(np.mean(np.array(up_calls['outofsample_prediction']-up_calls['barrier_price'])**2)))
print(RMSE)
MAE = float(np.average(np.abs(np.array(up_calls['outofsample_prediction']-up_calls['barrier_price']))))
MAE

In [ ]:
up_puts = outsample.copy()[
    (
        (outsample['w'] == 'put')
        &
        (outsample['updown'] =='Up')
    )
]
observed_price = np.sum(up_puts['barrier_price'])
outofsample_prediction = np.sum(up_puts['outofsample_prediction'])
print(f"\n{observed_price}\n{outofsample_prediction}\n")
RMSE = float(np.sqrt(np.mean(np.array(up_puts['outofsample_prediction']-up_puts['barrier_price'])**2)))
print(RMSE)
MAE = float(np.average(np.abs(np.array(up_puts['outofsample_prediction']-up_puts['barrier_price']))))
MAE